In [1]:
import pyGM as gm
import numpy as np
import matplotlib.pyplot as plt   # use matplotlib for plotting with inline plots
%matplotlib inline                

Let's define the Burglar & Earthquake model from lecture:

In [2]:
J,M,A,E,B = tuple(gm.Var(i,2) for i in range(0,5))  # all binary variables
X = [J,M,A,E,B]   # we'll often refer to variables as Xi or X[i]

# sometimes it's useful to have a reverse look-up from ID to "name string" (e.g. when drawing the graph)
IDtoName = dict( (eval(n).label,n) for n in ['J','M','A','E','B'])

pE = gm.Factor([E], [.998, .002]) # probability of earthquake (false,true)

pB = gm.Factor([B], [.999, .001]) # probability of burglary

pAgEB = gm.Factor([A,E,B], 0.0)   
# Set A,E,B                       # Note: it's important to refer to tuples like
pAgEB[:,0,0] = [.999, .001]       #  (A,E,B)=(0,0,0) in the order of the variables' ID numbers
pAgEB[:,0,1] = [.710, .290]       #  So, since A=X[2], E=X[3], etc., A,E,B is the correct order
pAgEB[:,1,0] = [.060, .940]
pAgEB[:,1,1] = [.050, .950]       # ":" refers to an entire row of the table

pJgA = gm.Factor([J,A], 0.0)      # Probability that John calls given the alarm's status
pJgA[:,0]    = [.95, .05]
pJgA[:,1]    = [.10, .90]

pMgA = gm.Factor([M,A], 0.0)      # Probability that Mary calls given the alarm's status
pMgA[:,0]    = [.99, .01]
pMgA[:,1]    = [.30, .70]

#factors = [pE, pB, pAgEB, pJgA, pMgA]  # collect all the factors that define the model
factors = [pJgA, pMgA, pAgEB, pE, pB]  # collect all the factors that define the model

fg = gm.GraphModel(factors)

## A-Star search for the MAP configuration
Let's find the most likely configuration using a simple fixed order a-star search:

In [3]:
import heapq
class PriorityQueue:
    '''Priority queue wrapper; returns highest priority element.'''
    def __init__(self): self.elements = []
    def __len__(self):  return len(self.elements)
    def empty(self):    return len(self.elements) == 0
    def push(self, item, pri): heapq.heappush(self.elements, (-pri, item))
    def pop(self):      return heapq.heappop(self.elements)[1]

def astar(model, order):
    '''Basic A-star search for graphical model 'model' using search order 'order'.'''
    def heur(model,config):
        return sum([np.log(f.condition(config).max()) for f in model.factors]);
    frontier = PriorityQueue()
    frontier.push({}, heur(model,{}))
    while frontier:
        current = frontier.pop()
        if len(current) == len(model.X): break   # if a full configuration, done:

        Xi = order[len(current)]         # get the next variable to assign (fixed order)
        for xi in range(Xi.states):      # generate each child node (assignment Xi=xi)
            next = current.copy();
            next[Xi] = xi;               # and add it to the frontier / queue
            frontier.push( next, heur(model,next) )

    return model.logValue(current), current      # found a leaf: return value & config

In [5]:
print( astar(fg, fg.X) )

(-0.06534663357889217, {Var (0,2): 0, Var (1,2): 0, Var (2,2): 0, Var (3,2): 0, Var (4,2): 0})


We can verify that this is indeed the value of the most probable configuration via variable elimination:

In [6]:
fg_copy = fg.copy()              # make a deep copy (VE changes the graph & factors)
fg_copy.eliminate(fg.X, 'max')   # eliminate via "max" over order 0...n-1
print(fg_copy.logValue([]))      # no variables left: only one (scalar) factor

-0.06534663357889212
